In [3]:
import numpy as np
import scipy.io.wavfile as wav
from scipy.signal import convolve
import matplotlib.pyplot as plt

In [4]:
def load_audio(file_path):
    # Load the audio file
    rate, data = wav.read(file_path)
    return data, rate

In [5]:
# Load and process audio file

# file_path = 'C:/Users/mrjac/Desktop/丁建均老師信號處理專題/thesis/hummingdata/10/lugo_愛你一萬年.wav'
file_path = 'C:/Users/mrjac/Desktop/丁建均老師信號處理專題/thesis/hummingdata/15/15lugo_自由.wav'
# file_path = 'C:/Users/mrjac/Desktop/丁建均老師信號處理專題/thesis/hummingdata/15/15bow_情非得已.wav'
# file_path = 'C:/Users/mrjac/Desktop/丁建均老師信號處理專題/thesis/hummingdata/20/20lugo_挪威的森林.wav'

data, rate = load_audio(file_path)
time_slot_width = 0.01

Step0 : Original Signal


In [ ]:
time = np.arange(data.shape[0]) / rate
plt.figure(figsize=(15, 3))
plt.plot(time, data)
plt.title('Original Signal')

Step 1: Find envelope amplitude -> Time Slot的尺度變換問題

In [ ]:
def find_envelope_amplitude(data, time_slot_width, rate):
    n0 = int(time_slot_width * rate)
    envelope_amplitude = np.abs(data[:len(data) // n0 * n0].reshape(-1, n0)).mean(axis=1)
    # b, a = signal.butter(2, 0.1)
    # envelope_amplitude = signal.filtfilt(b, a, np.abs(data))
    
    # Plotting the envelope amplitude
    time_slots = np.arange(envelope_amplitude.shape[0]) * time_slot_width
    plt.figure(figsize=(15, 3))
    plt.plot(time_slots, envelope_amplitude)
    plt.title('Envelope Amplitude (Step 1)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()
    
    return envelope_amplitude

envelope_amplitude_vals = find_envelope_amplitude(data, time_slot_width, rate)

Step 2: Reduce the effect of the background noise

In [ ]:
def reduce_background_noise(envelope_amplitude, rho=0.02):
    reduced_noise_amplitude = np.maximum(envelope_amplitude - rho, 0)
    
    # Plotting the reduced noise amplitude
    time_slots = np.arange(reduced_noise_amplitude.shape[0]) * time_slot_width
    plt.figure()
    plt.figure(figsize=(15, 3))
    plt.plot(time_slots, reduced_noise_amplitude)
    plt.title('Reduced Background Noise (Step 2)')
    plt.xlabel('Time Slot Index')
    plt.ylabel('Amplitude')
    plt.show()
    
    return reduced_noise_amplitude

reduced_noise_amplitude = reduce_background_noise(envelope_amplitude_vals)

Step 3: Normalize the envelope amplitude

In [ ]:
def normalize_envelope(envelope_amplitude):
    normalized_amplitude = envelope_amplitude / (0.2 + 0.1*np.mean(envelope_amplitude))     #prob
    
    # Plotting the normalized envelope amplitude
    time_slots = np.arange(normalized_amplitude.shape[0]) * time_slot_width
    plt.figure()
    plt.figure(figsize=(15, 3))
    plt.plot(time_slots, normalized_amplitude)
    plt.title('Normalized Envelope Amplitude (Step 3)')
    plt.xlabel('Time Slot Index')
    plt.ylabel('Normalized Amplitude')
    plt.show()
    
    return normalized_amplitude

normalized_amplitude = normalize_envelope(reduced_noise_amplitude)

Step 4: Take the fractional power of the envelope amplitude

In [ ]:
def fractional_power_amplitude(envelope_amplitude, laMbda=0.7):
    fractional_amplitude = envelope_amplitude ** laMbda
    
    # Plotting the fractional power amplitude
    plt.figure()
    plt.figure(figsize=(15, 3))
    plt.plot(fractional_amplitude)
    plt.title('Fractional Power Amplitude (Step 4)')
    plt.xlabel('Time Slot Index')
    plt.ylabel('Amplitude')
    plt.show()
    
    return fractional_amplitude

fractional_amplitude = fractional_power_amplitude(normalized_amplitude)

Step 5: Convolution with the envelope match filter

In [ ]:
def convolution_with_filter(fractional_amplitude, time_slot_width):
    match_filter = [3, 3, 4, 4, -1, -1, -2, -2, -2, -2, -2, -2]
    filtered_signal = convolve(fractional_amplitude, match_filter, mode='same')
    
    # Plotting the convolution result
    time_slots = np.arange(filtered_signal.shape[0]) * time_slot_width
    plt.figure()
    plt.figure(figsize=(15, 3))
    plt.plot(time_slots, filtered_signal)
    plt.title('Convolution with Envelope Match Filter (Step 5)')
    plt.xlabel('Time (s)')
    plt.ylabel('Filtered Signal')
    plt.show()
    
    return filtered_signal

filtered_signal = convolution_with_filter(fractional_amplitude, time_slot_width)

Step 6: Thresholding to find the onsets -> 閾值的選擇問題

In [ ]:
def detect_onsets(filtered_signal, threshold=4.5, data=None, rate=None, time_slot_width=None):
    onsets = np.where(filtered_signal > threshold)[0]
    
    # Plotting the onsets on the original signal
    if data is not None and rate is not None and time_slot_width is not None:
        time = np.arange(data.shape[0]) / rate
        plt.figure()
        plt.figure(figsize=(15, 3))
        plt.plot(time, data)
        for onset in onsets:
            onset_time = onset * time_slot_width
            plt.axvline(x=onset_time, color='red', linestyle='--', label='Onset' if onset == onsets[0] else "")
        plt.title('Detected Onsets (Step 6)')
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.legend()
        plt.show()
    
    return onsets

onsets = detect_onsets(filtered_signal, data=data, rate=rate, time_slot_width=time_slot_width)

Refine Onset : (a)[0.2 < Onset < L - 0.2], (b)[interval > 0.15]

In [ ]:

def refine_onsets(onsets, rate, time_slot_width, min_interval=0.15, start_offset=0.2, end_offset=0.2, data_length=None):
    # Convert 0.2 seconds and 0.15 seconds to samples
    start_limit = int(start_offset / time_slot_width)
    end_limit = int((rate - end_offset) / time_slot_width)
    min_interval_samples = int(min_interval / time_slot_width)

    # Exclude onsets near start and end
    refined_onsets = [onset for onset in onsets if start_limit <= onset <= end_limit]

    # Remove onsets that are too close to each other
    final_onsets = []
    for i in range(len(refined_onsets)):
        if i == 0 or (refined_onsets[i] - refined_onsets[i-1] >= min_interval_samples):
            final_onsets.append(refined_onsets[i])

    time = np.arange(data.shape[0]) / rate
    plt.figure()
    plt.figure(figsize=(15, 3))
    plt.plot(time, data)
    for onset in final_onsets:
        onset_time = onset * time_slot_width
        plt.axvline(x=onset_time, color='red', linestyle='--', label='Onset' if onset == final_onsets[0] else "")
    plt.title('Detected Onsets with Refine Process')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.show()

    return np.array(final_onsets)

refined_onsets = refine_onsets(onsets, rate, time_slot_width, data_length=len(data))